In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import tensorflow.keras.layers as layers
from sklearn.model_selection import KFold
import scipy
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from hyperopt import hp
from hyperopt import fmin, tpe, space_eval, rand
import seaborn as sns

In [2]:
DAY = 12
BATCH_SIZE = 128
WINDOW_OFFSET = 1 # DAY * 1
WINDOW_SIZE = DAY * 7

FIG_SIZE = (16, 10)
HYPERTUNE = True

In [3]:
def get_windows(x, win_in=100, win_off=50, skip=8):
    x = x[::skip]
    y = np.vstack([x[i+win_off:i+win_in+win_off] for i in range(len(x)-win_in-win_off)])
    x = np.vstack([x[i:i+win_in] for i in range(len(x)-win_in-win_off)])
    x = np.expand_dims(x, axis=-1)
    y = np.expand_dims(y, axis=-1)
    return x, y

def holidays_score(x, win_in=100, win_off=50, skip=8):
    x = x[::skip]
    return np.vstack([x[i+win_off:i+win_in+win_off] for i in range(len(x)-win_in-win_off)])

def standardize(x, mi, sigma):
    return (x.copy()-mi) / sigma

def preprocess_series(x, anomalies=None, factor=1.0):
    x, y = get_windows(x, win_in=WINDOW_SIZE, win_off=WINDOW_OFFSET)
    factor = int(factor * len(x))
    x, x_test = np.split(x, [factor])
    y, y_test = np.split(y, [factor])
    if anomalies is not None:
        anomalies = holidays_score(anomalies, win_in=WINDOW_SIZE, win_off=WINDOW_OFFSET)
        anomalies, anomalies_test = np.split(anomalies, [factor])
        return x, y, x_test, y_test, anomalies, anomalies_test
    return x, y, x_test, y_test

In [4]:
def get_model(units=[64, 32, 16, 8, 4], dropout_rate=0.1, inp_noise=0.0, inp_dropout=0.0, loss='mse', recurrent='gru', lr=1e-2, dense_activation='zero', hidden_unit=1):
    inp = layers.Input(shape=(None, 1))
    x = inp
    if recurrent != 'none':
        recurrent_layer = {
            'gru': layers.GRU,
            'lstm': layers.LSTM,
            'rnn': layers.SimpleRNN,
        }[recurrent]

        x = tf.keras.layers.GaussianNoise(inp_noise)(x)
        x = tf.keras.layers.Dropout(inp_dropout)(x)

        for unit in units:
            x = recurrent_layer(unit, return_sequences=True)(x)
            x = layers.Dropout(dropout_rate)(x)
            if dense_activation != 'zero':
                x = layers.Dense(hidden_unit)(x)
                x = layers.Activation(dense_activation)(x)
            

        x = layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs=inp, outputs=x)
    opt = tf.keras.optimizers.Adam(lr)
    loss = {'mse':'mse',
         'mae':'mae',
         'robust':tfa.losses.pinball_loss
    }[loss]
    model.compile(opt, loss, metrics=['mse', 'mape', 'mae'])
    return model


In [5]:
def fold_run(x, y, x_test, y_test, holidays=None, holidays_test=None, params={}, kfold=3, early_exit=-1,
             verbose=1, val_factor=0.4, batch_size=BATCH_SIZE):
    kf = KFold(n_splits=kfold)
    ans = {}
    ans['mean'] = []
    ans['cov'] = []
    ans['res_train'] = []
    ans['res_test'] = []
    ans['res_val1'] = []
    ans['res_val2'] = []
    ans['h_holidays_val2'] = []
    ans['h_holidays_test'] = []
    ans['y_train_pred'] = []
    ans['y_test_pred'] = []
    ans['y_val1_pred'] = []
    ans['y_val2_pred'] = []
    ans['y_train'] = []
    ans['y_test'] = []
    ans['y_val1'] = []
    ans['y_val2'] = []
    for idx, (train_index, val_index) in enumerate(kf.split(x)):
        if early_exit == idx:
            return ans
        print(len(train_index), len(val_index))
        print('run:', idx)
        model = get_model(**params)
        
        
        x_train = x[train_index]
        y_train = y[train_index]
        x_train_filtered = x_train[(holidays[train_index] == 0).all(axis=-1)]
        y_train_filtered = y_train[(holidays[train_index] == 0).all(axis=-1)]
        
        
        mi = np.mean(x_train_filtered)
        sigma = np.std(y_train_filtered)
        x_train_filtered = standardize(x_train_filtered, mi, sigma)
        y_train_filtered = standardize(y_train_filtered, mi, sigma)
        x_train = standardize(x_train, mi, sigma)
        y_train = standardize(y_train, mi, sigma)
        x_val = standardize(x[val_index], mi, sigma)
        y_val = standardize(y[val_index], mi, sigma)
        x_testn = standardize(x_test, mi, sigma)
        y_testn = standardize(y_test, mi, sigma)
        
        
        factor = int(val_factor * len(x_val))
        holidays_train = holidays[train_index]
        holidays_val1, holidays_val2 = np.split(holidays[val_index], [factor])
        x_val1, x_val2 = np.split(x_val, [factor])
        y_val1, y_val2 = np.split(y_val, [factor])
        x_val1 = x_val1[(holidays_val1 == 0).all(axis=-1)]
        y_val1 = y_val1[(holidays_val1 == 0).all(axis=-1)]
        
        assert holidays_val2.sum() > 0, 'No holidays in val2'

        hist = model.fit(x_train_filtered, y_train_filtered,
                 batch_size=batch_size,
                 epochs=1000,
                 verbose = verbose,
                 validation_data=(x_val1, y_val1),
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10),
                            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=3, factor=0.5, verbose=True)
        ]).history
        
        y_train_pred = model.predict(x_train, batch_size=2*BATCH_SIZE)
        y_test_pred = model.predict(x_testn, batch_size=2*BATCH_SIZE)
        y_val1_pred = model.predict(x_val1, batch_size=2*BATCH_SIZE)
        y_val2_pred = model.predict(x_val2, batch_size=2*BATCH_SIZE)
        
        # process s_*
        s_train = [[] for i in range(WINDOW_SIZE + len(y_train_pred))]
        s_test  = [[] for i in range(WINDOW_SIZE + len(y_test_pred))]
        s_val1   = [[] for i in range(WINDOW_SIZE + len(y_val1_pred))]
        s_val2   = [[] for i in range(WINDOW_SIZE + len(y_val2_pred))]
        
        for p1 in range(WINDOW_SIZE):
            for p0, val in enumerate(y_train_pred):
                s_train[p0 + p1].append(val[p1][0])
            for p0, val in enumerate(y_test_pred):
                s_test[p0 + p1].append(val[p1][0])
            for p0, val in enumerate(y_val1_pred):
                s_val1[p0 + p1].append(val[p1][0])
            for p0, val in enumerate(y_val2_pred):
                s_val2[p0 + p1].append(val[p1][0])
            
        s_train = np.array(s_train[WINDOW_SIZE:-WINDOW_SIZE])
        s_test = np.array(s_test[WINDOW_SIZE:-WINDOW_SIZE]) 
        s_val1 = np.array(s_val1[WINDOW_SIZE:-WINDOW_SIZE])
        s_val2 = np.array(s_val2[WINDOW_SIZE:-WINDOW_SIZE])
        
        # process h_*
        h_train = np.zeros(len(y_train)+WINDOW_SIZE)
        h_test  = np.zeros(len(y_testn)+WINDOW_SIZE)
        h_val1   = np.zeros(len(y_val1)+WINDOW_SIZE)
        h_val2   = np.zeros(len(y_val2)+WINDOW_SIZE)
        
        h_holidays_test = np.zeros(len(holidays_test)+WINDOW_SIZE) - 1
        h_holidays_val2 = np.zeros(len(holidays_val2)+WINDOW_SIZE) - 1

        for p1 in range(WINDOW_SIZE):
            for p0, val in enumerate(y_testn):
                assert(h_test[p0 + p1] == 0 or h_test[p0 + p1] == val[p1][0])
                h_test[p0 + p1] = val[p1][0]
            for p0, val in enumerate(y_val1):
                if h_val1[p0 + p1] == 0:
                    h_val1[p0 + p1] = val[p1][0]
            for p0, val in enumerate(y_val2):
                assert(h_val2[p0 + p1] == 0 or h_val2[p0 + p1] == val[p1][0])
                h_val2[p0 + p1] = val[p1][0]

            for p0, val in enumerate(holidays_val2):
                assert(h_holidays_val2[p0 + p1] == -1 or h_holidays_val2[p0 + p1] == val[p1])
                h_holidays_val2[p0 + p1] = val[p1]
            for p0, val in enumerate(holidays_test):
                assert(h_holidays_test[p0 + p1] == -1 or h_holidays_test[p0 + p1] == val[p1])
                h_holidays_test[p0 + p1] = val[p1]
            
        h_train = np.array(h_train[WINDOW_SIZE:-WINDOW_SIZE]).reshape(-1, 1)
        h_test = np.array(h_test[WINDOW_SIZE:-WINDOW_SIZE]).reshape(-1, 1)
        h_val1 = np.array(h_val1[WINDOW_SIZE:-WINDOW_SIZE]).reshape(-1, 1)
        h_val2 = np.array(h_val2[WINDOW_SIZE:-WINDOW_SIZE]).reshape(-1, 1)
        
        h_holidays_val2 = np.array(h_holidays_val2[WINDOW_SIZE:-WINDOW_SIZE])
        h_holidays_test = np.array(h_holidays_test[WINDOW_SIZE:-WINDOW_SIZE])
        
        res_train = h_train - s_train
        res_test = h_test - s_test
        res_val1 = h_val1 - s_val1
        res_val2 = h_val2 - s_val2
        
        for h in hist:
            if idx == 0:
                ans[h] = [min(hist[h])]
            else:
                ans[h].append(min(hist[h]))
        mean = np.mean(res_val1, axis=0)
        cov = np.cov(res_val1, rowvar=0)
        
        
        print('train:', len(res_train), 'holidays:', holidays_train.sum() / WINDOW_SIZE)
        print('test:', len(res_test),  'holidays:', holidays_test.sum() / WINDOW_SIZE)
        print('val1:', len(res_val1),  'holidays:', 0)
        print('val2:', len(res_val2),  'holidays:', holidays_val2.sum() / WINDOW_SIZE)
        
        ans['mean'].append(mean)
        ans['cov'].append(cov)
        ans['res_train'].append(res_train)
        ans['res_test'].append(res_test)
        ans['res_val1'].append(res_val1)
        ans['res_val2'].append(res_val2)
        ans['h_holidays_val2'].append(h_holidays_val2)
        ans['h_holidays_test'].append(h_holidays_test)
        ans['y_train_pred'].append(y_train_pred)
        ans['y_test_pred'].append(y_test_pred)
        ans['y_val1_pred'].append(y_val1_pred)
        ans['y_val2_pred'].append(y_val2_pred)
        ans['y_train'].append(y_train)
        ans['y_test'].append(y_testn)
        ans['y_val1'].append(y_val1)
        ans['y_val2'].append(y_val2)
        print()

    return ans

In [6]:
def print_results(ans, use_val=True, print_resp=True):
    thresholds = []
    zip_it = zip(ans['res_val2'], ans['res_test'], ans['mean'], ans['cov'], ans['h_holidays_val2'], ans['h_holidays_test'])
    resp = {'f0.1':[], 'threshold': [], 'cm':[], 'accuracy':[], 'recall': [], 'auc':[], 'precision':[]}
    for idx, (res_val, res_test, mean, cov, holidays_val, holidays_test) in enumerate(zip_it):
        if not use_val:
            res_val = res_test
            holidays_val = holidays_test
        
        prob = scipy.stats.multivariate_normal.pdf(res_val, mean=mean, cov=cov, allow_singular=True)
        
        threshold = 0
        threshold_i = -1
        bestf0_1 = 0
        prob_sorted = np.sort(prob.copy())
        best_conf_matrix = -1
        step = min(10000, len(prob_sorted))
        for i in range(step):
            p = prob_sorted[int(round(i/step*len(prob_sorted)))]
            y_pred = (prob < p).astype('int8')
            f0_1 = metrics.fbeta_score(holidays_val, y_pred, beta=0.1)
            if bestf0_1 < f0_1:
                bestf0_1 = f0_1
                threshold = p
                threshold_i = i
        
        prob = scipy.stats.multivariate_normal.pdf(res_test, mean=mean, cov=cov, allow_singular=True)
        y_pred = (prob < threshold).astype('int8')
        f0_1 = metrics.fbeta_score(holidays_test, y_pred, beta=0.1)
        acc = metrics.accuracy_score(holidays_test, y_pred)
        recall = metrics.recall_score(holidays_test, y_pred)
        precision = metrics.precision_score(holidays_test, y_pred)
        auc = metrics.roc_auc_score(holidays_test, y_pred)
        
        cm = pd.DataFrame(metrics.confusion_matrix(holidays_test, y_pred))
        
        cm.index.name = 'Reale'
        cm.columns.name = 'Previsto'
        resp['auc'].append(auc)
        resp['f0.1'].append(f0_1)
        resp['accuracy'].append(acc)
        resp['recall'].append(recall)
        resp['precision'].append(precision)
        resp['threshold'].append(threshold)
        resp['cm'].append(cm)
        
        if print_resp:
            print(f'idx: {idx} - threshold: {threshold} ({threshold_i}/{len(prob_sorted)}) - f0.1: {f0_1} - f0.1(validation): {bestf0_1}')
            print({name:resp[name][-1] for name in resp if name not in ['threshold', 'f0.1', 'cm']})
            print()
            print(metrics.classification_report(holidays_test, y_pred))
            sns.heatmap(cm, cmap="Blues", annot=True,annot_kws={"size": 16})
            plt.show()
            print()

    if not print_resp:
        return resp

In [7]:
def parse_params(params):
    params['units'] = []
    
    for i in range(3):
        if i < params['n_layers']:
            params['units'].append(int(params[f'unit_{i}']))
        del params[f'unit_{i}']
    del params['n_layers']
    del params['factor']
    del params['batch_size']
    return params

def objective(params):
    global x, y
    global holidays
    
    ans = {}
    factor = params['factor']
    batch_size = int(2**params['batch_size'])
    params = parse_params(params)
    split = 2000
    print(f'holidays: {holidays[split:].mean()} - holidays test: {holidays[:split].mean()}')
    ans = fold_run(x[split:], y[split:], x[:split], y[:split], holidays[split:], holidays[:split], early_exit=2, verbose=2, kfold=3, val_factor=factor)
    resp = print_results(ans, use_val=True, print_resp=False)
    print('params:', params, 'batch_size:', batch_size, 'factor:', factor, 'resp:', resp)
    return -np.mean(resp['f0.1']) # -np.mean(resp['f0.1'])

space = {'n_layers':hp.quniform('n_layers', 1, 2, 1),
         'unit_0':hp.quniform('unit_0', 16, 64, 1),
         'unit_1':hp.quniform('unit_1', 16, 64, 1),
         'unit_2':hp.quniform('unit_2', 16, 64, 1),
         'batch_size':hp.uniform('batch_size', 1, 10),
         'dropout_rate':hp.uniform('dropout_rate', 0.0, 0.3), 
         'hidden_unit': hp.quniform('hidden_unit', 1, 8, 1),
         # 'inp_noise':hp.uniform('inp_noise', 0.0, 0.1),
         # 'inp_dropout':hp.uniform('inp_dropout', 0.0, 0.1),
         # 'lr':hp.loguniform('lr', -5, -0.5),
         'dense_activation': hp.choice('dense_activation', ['swish', 'sigmoid']),
         'loss':hp.choice('loss', ['mse', 'mae', 'robust']),
         'recurrent':hp.choice('recurrent', ['lstm']),
         'factor': hp.uniform('factor', 0.45, 0.55),
}

# Modello

In [8]:
params = {
 'dropout_rate': 0.0,
 'inp_dropout': 0.0, 
 'inp_noise': 0.0,
 'loss': 'robust',
 'lr': 1e-2,
 'dense_activation': 'swish',
 'recurrent': 'lstm',
 'units': [64, 32],
 'hidden_unit': 4,
}
# params = {'dense_activation': 'swish', 'dropout_rate': 0.1, 'loss': 'mse', 'recurrent': 'lstm', 'units': [30, 20]}

In [9]:
get_model(**params).summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, None, 1)           0         
_________________________________________________________________
dropout (Dropout)            (None, None, 1)           0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 64)          0         
_________________________________________________________________
dense (Dense)                (None, None, 4)           260       
_________________________________________________________________
activation (Activation)      (None, None, 4)          

# Second Floor

In [10]:
df = pd.read_pickle('train1.pkl')
df

,AvgP1,AvgT,AvgH,AvgP2,AvgP3,DateTime,holidays,dayofweek
DateTime,,,,,,,,
2017-02-08 00:00:00,35.0,4.30,94.199997,34.0,87.0,2017-02-08 00:00:00,0,2
2017-02-08 00:15:00,0.0,4.80,94.550003,0.0,0.0,2017-02-08 00:15:00,0,2
2017-02-08 00:30:00,0.0,5.40,94.800003,51.0,0.0,2017-02-08 00:30:00,0,2
2017-02-08 00:45:00,0.0,5.40,94.449997,0.0,83.0,2017-02-08 00:45:00,0,2
2017-02-08 01:00:00,53.0,5.50,94.199997,0.0,0.0,2017-02-08 01:00:00,0,2
...,...,...,...,...,...,...,...,...
2019-04-09 22:45:00,41.0,10.95,83.250000,41.0,65.0,2019-04-09 22:45:00,0,1
2019-04-09 23:00:00,42.0,10.80,84.599998,42.0,68.0,2019-04-09 23:00:00,0,1
2019-04-09 23:15:00,43.0,10.65,86.050003,43.0,72.0,2019-04-09 23:15:00,0,1


In [11]:
x, y, x_test, y_test, holidays, holidays_test = preprocess_series(df['AvgP2'], df['holidays'])

x.shape, y.shape, x_test.shape, y_test.shape, holidays.shape, holidays_test.shape

((9407, 84, 1), (9407, 84, 1), (0, 84, 1), (0, 84, 1), (9407, 84), (0, 84))

In [ ]:
if HYPERTUNE:
    best = fmin(objective, space, algo=tpe.suggest, max_evals=100)
else:
    best = None
best

holidays: 0.024301336573511544 - holidays test: 0.006                                                                  
4938                                                                                                                   
2469                                                                                                                   
run:                                                                                                                   
0                                                                                                                      
Epoch 1/1000                                                                                                           
35/35 - 3s - loss: 0.4521 - mse: 0.4521 - mape: 205.5638 - mae: 0.5092 - val_loss: 0.1355 - val_mse: 0.1355 - val_mape: 89.1721 - val_mae: 0.2635

Epoch 2/1000                                                                                                           
35/35 - 1s - 

holidays:                                                                                                              
96.0                                                                                                                   
test:                                                                                                                  
1916                                                                                                                   
holidays:                                                                                                              
12.0                                                                                                                   
val1:                                                                                                                  
1133                                                                                                                   
holidays:                               

train:                                                                                                                 
4854                                                                                                                   
holidays:                                                                                                              
144.0                                                                                                                  
test:                                                                                                                  
1916                                                                                                                   
holidays:                                                                                                              
12.0                                                                                                                   
val1:                                   

35/35 - 1s - loss: 0.0930 - mse: 0.0930 - mape: 93.4219 - mae: 0.2189 - val_loss: 0.0482 - val_mse: 0.0482 - val_mape: 52.2632 - val_mae: 0.1426

Epoch 18/1000                                                                                                          
                                                                                                                       
Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
35/35 - 1s - loss: 0.0893 - mse: 0.0893 - mape: 88.9548 - mae: 0.2135 - val_loss: 0.0491 - val_mse: 0.0491 - val_mape: 48.0206 - val_mae: 0.1476

Epoch 19/1000                                                                                                          
35/35 - 1s - loss: 0.0866 - mse: 0.0866 - mape: 88.0870 - mae: 0.2102 - val_loss: 0.0476 - val_mse: 0.0476 - val_mape: 48.6610 - val_mae: 0.1422

Epoch 20/1000                                                                                                          
35/

Epoch 3/1000                                                                                                           
35/35 - 1s - loss: 0.1640 - mse: 0.1640 - mape: 160.6866 - mae: 0.2961 - val_loss: 0.1121 - val_mse: 0.1121 - val_mape: 121.7243 - val_mae: 0.2383

Epoch 4/1000                                                                                                           
35/35 - 1s - loss: 0.1454 - mse: 0.1454 - mape: 142.7313 - mae: 0.2760 - val_loss: 0.1045 - val_mse: 0.1045 - val_mape: 109.0396 - val_mae: 0.2228

Epoch 5/1000                                                                                                           
35/35 - 1s - loss: 0.1350 - mse: 0.1350 - mape: 136.1030 - mae: 0.2638 - val_loss: 0.1019 - val_mse: 0.1019 - val_mape: 103.0859 - val_mae: 0.2195

Epoch 6/1000                                                                                                           
35/35 - 1s - loss: 0.1265 - mse: 0.1265 - mape: 129.9463 - mae: 0.2543 - val

...

Epoch 19/1000                                                                                                          
                                                                                                                       
Epoch 00019: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
35/35 - 1s - loss: 0.0874 - mse: 0.0874 - mape: 97.0404 - mae: 0.2090 - val_loss: 0.0909 - val_mse: 0.0909 - val_mape: 89.8694 - val_mae: 0.2028

Epoch 20/1000                                                                                                          
35/35 - 1s - loss: 0.0848 - mse: 0.0848 - mape: 93.5705 - mae: 0.2053 - val_loss: 0.0905 - val_mse: 0.0905 - val_mape: 85.8390 - val_mae: 0.1995

Epoch 21/1000                                                                                                          
35/35 - 1s - loss: 0.0832 - mse: 0.0832 - mape: 92.6890 - mae: 0.2034 - val_loss: 0.0942 - val_mse: 0.0942 - val_mape: 88.3045 - val_mae: 0.2055

Epo

Epoch 7/1000                                                                                                           
35/35 - 1s - loss: 0.1149 - mse: 0.1149 - mape: 107.3529 - mae: 0.2440 - val_loss: 0.0590 - val_mse: 0.0590 - val_mape: 61.4983 - val_mae: 0.1632

Epoch 8/1000                                                                                                           
35/35 - 1s - loss: 0.1095 - mse: 0.1095 - mape: 103.2783 - mae: 0.2375 - val_loss: 0.0541 - val_mse: 0.0541 - val_mape: 55.9680 - val_mae: 0.1489

Epoch 9/1000                                                                                                           
35/35 - 1s - loss: 0.1048 - mse: 0.1048 - mape: 100.2128 - mae: 0.2319 - val_loss: 0.0516 - val_mse: 0.0516 - val_mape: 58.5314 - val_mae: 0.1511

Epoch 10/1000                                                                                                          
35/35 - 1s - loss: 0.1009 - mse: 0.1009 - mape: 97.7068 - mae: 0.2279 - val_los

4938                                                                                                                   
2469                                                                                                                   
run:                                                                                                                   
1                                                                                                                      
Epoch 1/1000                                                                                                           
35/35 - 3s - loss: 0.5205 - mse: 0.5205 - mape: 238.0217 - mae: 0.5545 - val_loss: 0.1564 - val_mse: 0.1564 - val_mape: 146.7496 - val_mae: 0.2778

Epoch 2/1000                                                                                                           
35/35 - 1s - loss: 0.2082 - mse: 0.2082 - mape: 180.2592 - mae: 0.3330 - val_loss: 0.1295 - val_mse: 0.1295 - val_mape: 131.1794 - v

val1:                                                                                                                  
951                                                                                                                    
holidays:                                                                                                              
0                                                                                                                      
val2:                                                                                                                  
1255                                                                                                                   
holidays:                                                                                                              
24.0                                                                                                                   
params:                                 

Epoch 20/1000                                                                                                          
35/35 - 1s - loss: 0.0836 - mse: 0.0836 - mape: 85.7842 - mae: 0.2066 - val_loss: 0.0503 - val_mse: 0.0503 - val_mape: 52.2745 - val_mae: 0.1443

Epoch 21/1000                                                                                                          
35/35 - 1s - loss: 0.0831 - mse: 0.0831 - mape: 85.8087 - mae: 0.2057 - val_loss: 0.0487 - val_mse: 0.0487 - val_mape: 49.7523 - val_mae: 0.1434

Epoch 22/1000                                                                                                          
                                                                                                                       
Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
35/35 - 1s - loss: 0.0818 - mse: 0.0818 - mape: 84.8954 - mae: 0.2043 - val_loss: 0.0503 - val_mse: 0.0503 - val_mape: 49.9492 - val_mae: 0.1475

Ep

In [ ]:
if HYPERTUNE:
    params = parse_params(space_eval(space, best))
    print(best)
    print(params)
    

In [ ]:
ans = fold_run(x, y, x_test, y_test, holidays, holidays_test, params=params)

In [ ]:
for metric in ['mse', 'val_mse', 'mae', 'val_mae', 'mape', 'val_mape']:
    print(f"{metric.rjust(8)}: {round(np.mean(ans[metric]), 3)} +- {round(np.std(ans[metric]), 3)}")

In [ ]:
#convariance matrix
for idx, cov in enumerate(ans['cov']):
    print(f'{idx} - rank: {np.linalg.matrix_rank(cov)}/{len(cov)} det: {np.linalg.det(cov)} min(eigvals): {np.min(np.linalg.eigvals(cov))}')

fig, axs = plt.subplots(ncols=len(ans['cov']), figsize=FIG_SIZE)
if len(ans['cov']) == 1:
    axs = [axs]
for ax, cov in zip(axs, ans['cov']):
    cax = ax.matshow(cov)
    ax.grid()
cbar = fig.colorbar(cax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=FIG_SIZE)
kfold = 0
prob_normaldays = scipy.stats.multivariate_normal.pdf(ans['res_test'][kfold][ans['h_holidays_test'][kfold] == 0], mean=ans['mean'][kfold], cov=ans['cov'][0], allow_singular=True)
prob_holidays = scipy.stats.multivariate_normal.pdf(ans['res_test'][kfold][ans['h_holidays_test'][kfold] == 1], mean=ans['mean'][kfold], cov=ans['cov'][0], allow_singular=True)

ax.hist(np.clip(np.log(prob_normaldays+1e-30), -100, 300), label='log of pdf of normal days', bins='auto', density=True, alpha=0.5)
ax.hist(np.clip(np.log(prob_holidays+1e-30), -100, 300), label='log of pdf of holidays', bins='auto', density=True, alpha=0.5)

ax.grid()
ax.legend()
plt.show()

In [ ]:
#mean graph
fig, ax = plt.subplots(ncols=1, figsize=FIG_SIZE)

for idx, mean in enumerate(ans['mean']):
    ax.plot(mean, label=f'mean {idx}')

ax.grid()
ax.legend()
plt.show()

In [ ]:
best_idx = np.argmin(ans['mean'][0])
best_idx, np.argsort(ans['mean'][0])

In [ ]:
## test data
kfold = 0
lower = DAY * 63
upper = lower + DAY * 28
OFF = best_idx - WINDOW_SIZE # + WINDOW_OFFSET
fig, ax = plt.subplots(figsize=FIG_SIZE)
ax.plot([i[best_idx] for i in ans['y_test'][kfold][lower:upper]], label='Reale')
ax.plot([i[best_idx] for i in ans['y_test_pred'][kfold][lower:upper]], label='Previsto')

try:
    prob = scipy.stats.multivariate_normal.pdf(ans['res_test'][kfold][lower+OFF:upper+OFF], mean=ans['mean'][kfold], cov=ans['cov'][kfold], allow_singular=True)
    split_factor = 1 / prob[np.argsort(prob)[int(len(prob) * 0.04)]]
    prob = np.clip(split_factor * prob, 1.0, 2)
    ax.plot(prob, label='Probabilità', linestyle='dashed')
except Exception as e:
    print('Error:', e)

ax.plot(ans['h_holidays_test'][kfold][lower+OFF:upper+OFF], label='Anomalie', alpha=0.5)

# ax.plot([i[best_idx] - j[best_idx] for i, j in zip(ans['y_test'][0][lower:upper], ans['y_test_pred'][kfold][lower:upper])], label='Errore', alpha=0.5)
# ax.plot([i[best_idx] for i in ans['res_test'][kfold][lower+OFF:upper+OFF]], label='Errore off', alpha=0.5)

ax.grid()
ax.legend()
ax.set_title('Test Data')
plt.show()

In [ ]:
# training data
kfold = 0
lower = DAY * 35
upper = lower + DAY * 28
OFF = best_idx - WINDOW_SIZE # + WINDOW_OFFSET
fig, ax = plt.subplots(figsize=FIG_SIZE)
ax.plot([i[best_idx] for i in ans['y_train'][kfold][lower:upper]], label='Reale')
ax.plot([i[best_idx] for i in ans['y_train_pred'][kfold][lower:upper]], label='Previsto')
try:
    prob = scipy.stats.multivariate_normal.pdf(ans['res_train'][kfold][lower+OFF:upper+OFF], mean=ans['mean'][kfold], cov=ans['cov'][kfold], allow_singular=True)
    split_factor = 1 / prob[np.argsort(prob)[int(len(prob) * 0.04)]]
    prob = np.clip(split_factor * prob, 1.0, 2)
    ax.plot(prob, label='Probabilità', linestyle='dashed')
except Exception as e:
    print('Error:', e)
ax.set_title('Training Data')
ax.grid()
ax.legend()
plt.show()

In [ ]:
# precision - recall - F0.1 - accuracy
print_results(ans, True)

# Third Floor

In [ ]:
df = pd.read_pickle('train1.pkla')

In [ ]:
x, y, x_test, y_test, holidays, holidays_test = preprocess_series(df['AvgP3'], df['holidays'])

x.shape, y.shape, x_test.shape, y_test.shape, holidays.shape, holidays_test.shape

In [ ]:
ans = fold_run(x, y, x_test, y_test, holidays, holidays_test, params=params)

In [ ]:
for metric in ['mse', 'val_mse', 'mae', 'val_mae', 'mape', 'val_mape']:
    print(f"{metric.rjust(8)}: {round(np.mean(ans[metric]), 3)} +- {round(np.std(ans[metric]), 3)}")

In [ ]:
lower = 12 * 0
upper = lower + 12 * 28
fig, ax = plt.subplots(figsize=FIG_SIZE)
ax.plot([i[best_idx] for i in ans['y_test'][0][lower:upper]], label='Reale')
ax.plot([i[best_idx] for i in ans['y_test_pred'][0][lower:upper]], label='Previsto')

ax.grid()
ax.legend()
plt.show()

In [ ]:
print_results(ans, True)

# Olanda

In [ ]:
with open('power_data_eamonn-1.txt', 'r') as f:
    x = f.readlines()
x = np.array([int(i) for i in x if i != '\n'])

In [ ]:
x, y, x_test, y_test = preprocess_series(x)
x.shape, y.shape, x_test.shape, y_test.shape

In [ ]:
ans = fold_run(x, y, x_test, y_test, params=params)

In [ ]:
for metric in ['mse', 'val_mse', 'mae', 'val_mae', 'mape', 'val_mape']:
    print(f"{metric.rjust(8)}: {round(np.mean(ans[metric]), 3)} +- {round(np.std(ans[metric]), 3)}")

In [ ]:
best_idx = np.argmin(abs(ans['mean'][0]))
best_idx

In [ ]:
lower = DAY * 50
upper = lower + DAY * 21
fig, ax = plt.subplots(figsize=FIG_SIZE)
ax.plot([i[best_idx] for i in ans['y_test'][0][lower:upper]], label='Reale')
ax.plot([i[best_idx] for i in ans['y_test_pred'][0][lower:upper]], label='Previsto')

ax.grid()
ax.legend()
plt.show()